In [2]:
import casadi as ca
import numpy as np
import math

In [3]:
roll = ca.SX.sym('roll')
pitch = ca.SX.sym('pitch')
yaw = ca.SX.sym('yaw')

r_roll = ca.SX_eye(3)
r_roll[1:,1:] = np.array([[ca.cos(roll), -ca.sin(roll)],
                 [ca.sin(roll), ca.cos(roll)]])
r_pitch = ca.SX_eye(3)
r_pitch[[0,2],[0,2]] = np.array([[ca.cos(pitch), ca.sin(pitch)],
                 [-ca.sin(pitch), ca.cos(pitch)]])
r_yaw = ca.SX_eye(3)
r_yaw[:2,:2] = np.array([[ca.cos(yaw), -ca.sin(yaw)],
                 [ca.sin(yaw), ca.cos(yaw)]])

print("ROLL", r_roll)
print("PITCH", r_pitch)
print("YAW", r_yaw)

R = (r_yaw @ r_pitch @ r_roll)
print("Y-P-R")
for i in range(0,3):
    for j in range(0,3):
        print(R[i, j],end=' , ')
    print()

print(ca.Function('rot', [yaw, pitch], [r_yaw @ r_pitch])(11*math.pi/180, 68*math.pi/180))


ROLL 
[[1, 00, 00], 
 [00, cos(roll), (-sin(roll))], 
 [00, sin(roll), cos(roll)]]
PITCH 
[[cos(pitch), 00, sin(pitch)], 
 [00, 1, 00], 
 [(-sin(pitch)), 00, cos(pitch)]]
YAW 
[[cos(yaw), (-sin(yaw)), 00], 
 [sin(yaw), cos(yaw), 00], 
 [00, 00, 1]]
Y-P-R
(cos(yaw)*cos(pitch)) , (((cos(yaw)*sin(pitch))*sin(roll))-(sin(yaw)*cos(roll))) , ((sin(yaw)*sin(roll))+((cos(yaw)*sin(pitch))*cos(roll))) , 
(sin(yaw)*cos(pitch)) , ((cos(yaw)*cos(roll))+((sin(yaw)*sin(pitch))*sin(roll))) , (((sin(yaw)*sin(pitch))*cos(roll))-(cos(yaw)*sin(roll))) , 
(-sin(pitch)) , (cos(pitch)*sin(roll)) , (cos(pitch)*cos(roll)) , 

[[0.367724, -0.190809, 0.910149], 
 [0.0714783, 0.981627, 0.176915], 
 [-0.927184, 00, 0.374607]]


In [15]:
## For magnetometer measurement correction
print("Correction : Intrinsic P-R")
R = r_pitch @ r_roll
for i in range(0,3):
    for j in range(0,3):
        print(R[i, j],end=' , ')
    print()


Correction : Intrinsic P-R
cos(pitch) , (sin(pitch)*sin(roll)) , (sin(pitch)*cos(roll)) , 
00 , cos(roll) , (-sin(roll)) , 
(-sin(pitch)) , (cos(pitch)*sin(roll)) , (cos(pitch)*cos(roll)) , 


For our complementary filter we use an intrinsic Y-P-R sequence, meaning from the inertial frame we first rotate by a yaw angle $\psi$ then by a pitch angles $\theta$ then by a roll angle $\phi$ giving our rotation from inertial to body as $${}^{I}R_{B}=R_{\psi}R_{\theta}R_{\phi}$$
$$R=\begin{bmatrix}
\cos{\psi} & -\sin{\psi} & 0 \\
\sin{\psi} & \cos{\psi} & 0 \\
0 & 0 & 1 \\
\end{bmatrix}
\begin{bmatrix}
\cos{\theta} & 0 & \sin{\theta} \\
0 & 1 & 0 \\
-\sin{\theta} & 0 & \cos{\theta} \\
\end{bmatrix}
\begin{bmatrix}
1 & 0 & 0 \\
0 & \cos{\phi} & -\sin{\phi} \\
0 & \sin{\phi} & \cos{\phi}
\end{bmatrix}$$

Assuming no yaw ($\psi=0$), we know our tilt is defined as:
$$\begin{bmatrix}
0 \\
0 \\
-g \\
\end{bmatrix}=
\begin{bmatrix}
\cos{\theta} & \sin{\theta}\sin{\phi} & \sin{\theta}\cos{\phi} \\
0 & \cos{\phi} & -\sin{\phi} \\
-\sin{\theta} & \cos{\theta}\sin{\phi} & \cos{\theta}\cos{\phi} \\
\end{bmatrix}
\begin{bmatrix}
a_{x} \\
a_{y} \\
a_{z} \\
\end{bmatrix}
$$

TODO Add computation of roll,pitch accelerometer equations

Again, we can re-use the roll/pitch rotation to transform our body-frame magnetometer reading into a 'yaw-only' frame with x and y axes aligned to the NED frame X-Y plane
$$\begin{align}
\begin{bmatrix}
m_{x,comp} \\
m_{y,comp} \\
m_{z,comp} \\
\end{bmatrix}&=
\begin{bmatrix}
\cos{\theta} & \sin{\theta}\sin{\phi} & \sin{\theta}\cos{\phi} \\
0 & \cos{\phi} & -\sin{\phi} \\
-\sin{\theta} & \cos{\theta}\sin{\phi} & \cos{\theta}\cos{\phi} \\
\end{bmatrix}
\begin{bmatrix}
m_{x} \\
m_{y} \\
m_{z} \\
\end{bmatrix} \\
&=\begin{bmatrix}
m_{x}\cos{\theta} + m_{y}\sin{\theta}\sin{\phi} + m_{z}\sin{\theta}\cos{\phi} \\
m_{y}\cos{\phi} -m_{z}\sin{\phi} \\
-m_{x}\sin{\theta} + m_{y}\cos{\theta}\sin{\phi} + m_{z}\cos{\theta}\cos{\phi} \\
\end{bmatrix}
\end{align}
$$
In our 'yaw-only' frame, and ignoring magnetic declination for now, we can relate the magnetic field vector to our (tilt-compensated) reading via
$$
B_{ref}=
\begin{bmatrix}
b_{x} \\
0 \\
b_{z}
\end{bmatrix}
= \begin{bmatrix}
\cos{\psi} & -\sin{\psi} & 0 \\
\sin{\psi} & \cos{\psi} & 0 \\
0 & 0 & 1 \\
\end{bmatrix}
\begin{bmatrix}
m_{x,comp} \\
m_{y,comp} \\
m_{z,comp} \\
\end{bmatrix}
$$
Now our $B_{ref}$ vector has x and z components only and hence we can use the second equation to compute yaw
$$\psi=\text{atan2}(-m_{y,comp},m_{x,comp})$$

The last step is to offset this yaw measurement by the magnetic declination for the current location.

In [14]:
roll = ca.SX.sym('roll')
pitch = ca.SX.sym('pitch')
yaw = ca.SX.sym('yaw')

r_roll = ca.SX_eye(3)
r_roll[1:,1:] = np.array([[ca.cos(roll), -ca.sin(roll)],
                 [ca.sin(roll), ca.cos(roll)]])
r_pitch = ca.SX_eye(3)
r_pitch[[0,2],[0,2]] = np.array([[ca.cos(pitch), ca.sin(pitch)],
                 [-ca.sin(pitch), ca.cos(pitch)]])
r_yaw = ca.SX_eye(3)
r_yaw[:2,:2] = np.array([[ca.cos(yaw), -ca.sin(yaw)],
                 [ca.sin(yaw), ca.cos(yaw)]])

print("ROLL", r_roll)
print("PITCH", r_pitch)
print("YAW", r_yaw)

print(r_pitch @ r_roll)

ROLL 
[[1, 00, 00], 
 [00, cos(roll), (-sin(roll))], 
 [00, sin(roll), cos(roll)]]
PITCH 
[[cos(pitch), 00, sin(pitch)], 
 [00, 1, 00], 
 [(-sin(pitch)), 00, cos(pitch)]]
YAW 
[[cos(yaw), (-sin(yaw)), 00], 
 [sin(yaw), cos(yaw), 00], 
 [00, 00, 1]]
@1=sin(pitch), @2=sin(roll), @3=cos(pitch), @4=cos(roll), 
[[cos(pitch), (@1*@2), (@1*@4)], 
 [00, cos(roll), (-sin(roll))], 
 [(-sin(pitch)), (@3*@2), (@3*@4)]]


Our accelerometer reading is taken in the body frame. In this frame, we can use the accelerometer y and z readings to calculate the roll.
$$\phi_{a}=\text{atan2f}(-a_{y}, -a_{z})$$

Similarly, we can rotate to the 'pitch' frame by 